<a href="https://colab.research.google.com/github/Existanze54/sirius-neural-networks-2024/blob/main/Homeworks/HW4_GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 4. Собираем свой GAN

В этом задании необходимо будет воссоздать архитектуру GAN (Generative Adversarial Networks) модели, состоящую из генератора и дискриминатора. В качестве данных будем использовать стандартный датасете MNIST. Хотим иметь возможность генерировать качественные изображения рукописных цифр по запросу.


In [ ]:
%matplotlib inline
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

from torch.utils.data import DataLoader
from PIL import Image
from torch import autograd
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [ ]:
from torchvision.transforms import ToTensor, Normalize, Compose
from torchvision.datasets import MNIST # самый обычный MNIST

# Загрузка датасета

mnist = MNIST(root='data',
              train=True,
              download=True,
              transform=Compose([ToTensor(), Normalize(mean=(0.5,), std=(0.5,))]))

Помещаем все значения в диапазон от 0 до 1

In [ ]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

img, label = mnist[0]
print('Label: ', label)
img_norm = denorm(img)
plt.imshow(img_norm[0], cmap='gray')

In [ ]:
img.shape

In [ ]:
data_loader = torch.utils.data.DataLoader(mnist,
                                          batch_size=64,
                                          drop_last=True, # чтобы не было проблем с незаполненным последним батчем при работе с CNN
                                          shuffle=True)

# Задание 1. Простейший GAN
### Что такое GAN?

В 2014 году, [Goodfellow et al.](https://arxiv.org/abs/1406.2661) опубликовали метод для тренировки генеративных моделей, который называется Generative Adversarial Networks (GANs).

Давайте воспроизведем архитектуру из этой статьи.

### Для начала вспомним в общем архитектуру GAN.
Генератор создает "изображения" из случайного шума и подает результат на вход дискриминатора. Дискриминатор обучается на реальных изображениях и на изображениях, сгенерированных генератором, выдает свою оценку, и градиенты используются для обновления весов как дискриминатора, так и генератора. Помним, что дискриминатор может быть слишком строгим, поэтому его можно обучать реже, либо сделать чуть лояльнее к генератору, используя дропауты.

<img src="https://data.bioml.ru/htdocs/courses/bioml/neural_networks/gan/img/gan_scheme.png" alt="Drawing" width= "800px;"/>

Поскольку мы хотим иметь возможность генерировать изображения цифр по запросу, мы будем и генератору и дискриминатору передавать метку класса. То есть будем использовать архитектуру GAN с условиями (cGAN).


<img src="https://data.bioml.ru/htdocs/courses/bioml/neural_networks/gan/img/gan_conditional_scheme.png" alt="Drawing" width= "800px;"/>

### Дискриминатор
Реализуйте архитектуру дискриминатора по приведенной ниже схеме

1. Полносвязный линейный слой с инпутом размера: размер изображения (28*28) + размер эмбеддинга меток (например, 10); и выходом 256
2. LeakyReLU с alpha=0.01
3. Полносвязный линейный слой с выходом 256
4. LeakyReLU с alpha=0.01
5. Полносвязный линейный слой с выходом 1
6. Функция активации (какая?)

LeakyRelu возвращает $f(x) = \max(\alpha x, x)$ с некой константой $\alpha$; здесь коэффициент нелинейности равен $\alpha=0.01$.

Аутпут дискриминатора должен содержать число, соответствующее вероятности изображения быть подлинным.



In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        set_random_seed(42) # оставляем для воспроизводимости результатов
        super().__init__()

        # можно было закодировать one_hot, но мы захотели эмбеддинг меток
        # эмбеддинг просто дает более богатое представление и иногда позволяет сэкономить несколько слоев сети.
        # но здесь это не важно, будет работать что то, что то.
        # везде в дальнейшем оставляем эмбеддинг
        self.label_emb = nn.Embedding(10, 10)

        self.model = nn.Sequential(
            # your code here
        )

    def forward(self, x, labels):
        # В дискриминатор мы должны подать само изображение (28*28), склеенное с меткой
        x = x.view(x.size(0), 784)
        c = self.label_emb(labels)
        x = torch.cat([x, c], 1) # и дискриминатор и генератор получают на вход информацию о метке
        out = self.model(x)
        return out.squeeze()

### Генератор
Теперь соберем, собственно, генератор
1. Полносвязный линейный слой с инпутом размера: размер вектора шума (например, 100) + размер эмбеддинга меток (10); и выходом 1024
2. ReLU
3. Полносвязный линейный слой с выходом 1024
4. ReLU
5. Полносвязный линейный слой с выходом 784
6. TanH - чтобы значения пикселей лежали в пределах [-1,1]

In [ ]:
class Generator(nn.Module):
    def __init__(self, noise_dim=100):
        set_random_seed(42)
        super().__init__()

        self.label_emb = nn.Embedding(10, 10)

        self.model = nn.Sequential(
            # your code here
        )

    def forward(self, z, labels):
        c = self.label_emb(labels)
        x = torch.cat([z, c], 1) # и дискриминатор и генератор получают на вход информацию о метке
        out = self.model(x)
        return out.view(x.size(0), 28, 28) # разворачиваем линейный аутпут в картинку

В начале в качестве функции потерь будем использовать стандартный BCELoss

$$ bce(s, y) = y * \log(s) + (1 - y) * \log(1 - s) $$

In [ ]:
criterion = nn.BCELoss()

In [ ]:
generator = Generator().cuda()
discriminator = Discriminator().cuda()

В качестве оптимизатора будет использовать Adam с параметрами learning rate = 1e-3, betas = (0.5, 0.999)

In [ ]:
?torch.optim.Adam

In [ ]:
d_optimizer = # your code here

g_optimizer = # your code here

Когда мы учим дискриминатор, мы передаем ему реальные объекты с метками + сгенерированные фейковые объекты с фейковыми метками, считаем loss по всем этим объектам, делаем backprop до весов дискриминатора и шаг по антиградиенту


In [ ]:
def discriminator_train_step(batch_size, discriminator, generator, d_optimizer, criterion, real_images, labels):
    d_optimizer.zero_grad()

    # Подлинные изображения
    real_validity = discriminator(real_images, labels)
    real_loss = criterion(real_validity, torch.ones(batch_size).cuda())

    # Сгенерированные изображения
    z = torch.randn(batch_size, 100).cuda() # генерируем шум
    fake_labels = torch.LongTensor(np.random.randint(0, 10, batch_size)).cuda() # генерируем случайные метки для каждого изображения
    fake_images = generator(z, fake_labels) # собираем в картинку
    fake_validity = discriminator(fake_images, fake_labels) # дискриминатор будет сильно штрафовать генератор если тот сгенерил объект неправильно по метке.
    fake_loss = criterion(fake_validity, torch.zeros(batch_size).cuda())

    d_loss = real_loss + fake_loss
    # your code here
    # your code here
    return d_loss.data

Теперь настроим шаг обучения генератора. Так же генерируем фейковые картинки и отдаем дискриминатору на оценку, считаем loss между уверенностью дискриминатора и единицами (чтобы оценить, насколько генератору удается обмануть дискриминатор), делаем backprop до весов генератора сквозь всю модель.

In [ ]:
def generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion):
    g_optimizer.zero_grad()
    z = torch.randn(batch_size, 100).cuda()
    fake_labels = torch.LongTensor(np.random.randint(0, 10, batch_size)).cuda()
    fake_images = generator(z, fake_labels)
    validity = discriminator(fake_images, fake_labels)
    g_loss = criterion(validity, torch.ones(batch_size).cuda())
    # your code here (loss backward)
    # your code here (step optimizer)
    return g_loss.data

Тренируем наш GAN

In [ ]:
num_epochs = 30
display_step = 300
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch}...')
    for i, (images, labels) in enumerate(data_loader):
        real_images = images.cuda()
        labels = labels.cuda()
        generator.train()
        batch_size = real_images.size(0)
        d_loss = discriminator_train_step(len(real_images), discriminator,
                                          generator, d_optimizer, criterion,
                                          real_images, labels)


        g_loss = generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion)

    generator.eval()
    print(f'g_loss: {g_loss:.4f}, d_loss: {d_loss:.4f}')
    z = torch.randn(9, 100).cuda()
    labels = torch.LongTensor(np.arange(9)).cuda()
    sample_images = generator(z, labels).unsqueeze(1).data.cpu()
    grid = make_grid(sample_images, nrow=3, normalize=True).permute(1,2,0).numpy()
    plt.imshow(grid)
    plt.show()

Смотрим как генерируются изображения по запросу

In [ ]:
z = torch.randn(100, 100).cuda()
labels = torch.LongTensor([i for _ in range(10) for i in range(10)]).cuda()
sample_images = generator(z, labels).unsqueeze(1).data.cpu()
grid = make_grid(sample_images, nrow=10, normalize=True).permute(1,2,0).numpy()
fig, ax = plt.subplots(figsize=(15,15))
ax.imshow(grid)
_ = plt.yticks([])
_ = plt.xticks(np.arange(15, 300, 30), ['0', '1', '2', '3',\
                                        '4', '5', '6', '7', '8',\
                                        '9'], rotation=45, fontsize=20)

Сделайте вывод относительно качества генерируемых изображений такой архитектурой.

In [ ]:
# your conclusions here

# Задание 2. Least Squares GAN
Теперь мы рассмотрим [Least Squares GAN](https://arxiv.org/abs/1611.04076), новую, более стабильную альтернативу исходной функции потерь GAN.
LSGAN использует квадратичную функцию потерь вместо логарифмической, чтобы стабилизировать обучение и избежать проблем с градиентами.

В этой части нам нужно только изменить функцию потерь и переобучить модель. Мы будем реализовывать уравнение (9) из статьи, где функция потерь генератора имеет вид:

Функция потерь генератора:
$$\ell_G  =  \frac{1}{2}\mathbb{E}_{z \sim p(z)}\left[\left(D(G(z))-1\right)^2\right]$$
Здесь мы хотим, чтобы дискриминатор $D(G(z))$ оценивал сгенерированные данные $G(z)$ как реальные (или близкие к 1)

Функция потерь дискриминатора
$$ \ell_D = \frac{1}{2}\mathbb{E}_{x \sim p_\text{data}}\left[\left(D(x)-1\right)^2\right] + \frac{1}{2}\mathbb{E}_{z \sim p(z)}\left[ \left(D(G(z))\right)^2\right]$$
Теперь мы хотим, чтобы дискриминатор оценивал реальные данные $D(x)$ как подлинные, а аутпут генератора $D(G(z))$ - как фейк


**Замечание**: Вместо расчета матожидания $\mathbb{E}$ будем брать среднее значение по элементам минибатча `torch.mean`. В качестве результатов дискриминатора $D(x)$ и $D(G(z))$ используем прямой атупут генератора (`scores_real` and `scores_fake`).

In [ ]:
def ls_discriminator_loss(scores_real, scores_fake):
    """
    Compute the Least-Squares GAN loss for the discriminator.

    Inputs:
    - scores_real: PyTorch Variable of shape (N,) giving scores for the real data.
    - scores_fake: PyTorch Variable of shape (N,) giving scores for the fake data.

    Outputs:
    - loss: A PyTorch Variable containing the loss.
    """
    N = scores_real.size()
    loss = (0.5 * torch.mean((scores_real-torch.ones(N).cuda())**2)) + (0.5 * torch.mean(scores_fake**2))
    return loss

def ls_generator_loss(scores_fake):
    """
    Computes the Least-Squares GAN loss for the generator.

    Inputs:
    - scores_fake: PyTorch Variable of shape (N,) giving scores for the fake data.

    Outputs:
    - loss: A PyTorch Variable containing the loss.
    """
    N = scores_fake.size()
    loss = (0.5 * torch.mean((scores_fake-torch.ones(N).cuda())**2))
    return loss

In [ ]:
generator = Generator().cuda()
discriminator = Discriminator().cuda()

Добавим значения бетта 1 и бетта 2 к Адаму, без этого изменения дискриминатор очень быстро достигает значения 0 и генератор не учится

In [ ]:
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=1e-3, betas = (0.5, 0.999))
g_optimizer = torch.optim.Adam(generator.parameters(), lr=1e-3, betas = (0.5, 0.999))

In [ ]:
def ls_discriminator_train_step(batch_size, discriminator, generator, d_optimizer, criterion, real_images, labels):
    d_optimizer.zero_grad()

    # Тренировка на подлинных изображениях
    real_validity = discriminator(real_images, labels)

    # Тренировка на фейковых изображениях
    z = torch.randn(batch_size, 100).cuda()
    fake_labels = torch.LongTensor(np.random.randint(0, 10, batch_size)).cuda()
    fake_images = generator(z, fake_labels)
    fake_validity = discriminator(fake_images, fake_labels) # дискриминатор будет сильно штрафовать генератор если тот сгенерил объект неправильно по метке.

    d_loss = # your code here

    # your code here
    # your code here
    return d_loss.data

In [ ]:
def ls_generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion):
    g_optimizer.zero_grad()

    z = torch.randn(batch_size, 100).cuda()
    fake_labels = torch.LongTensor(np.random.randint(0, 10, batch_size)).cuda()
    fake_images = generator(z, fake_labels)
    validity = discriminator(fake_images, fake_labels)
    g_loss = # your code here
    # your code here
    # your code here
    return g_loss.data

In [ ]:
num_epochs = 30
display_step = 300
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch}...')
    for i, (images, labels) in enumerate(data_loader):
        real_images = images.cuda()
        labels = labels.cuda()
        generator.train()
        batch_size = real_images.size(0)
        d_loss = # your code here


        g_loss = # your code here

    generator.eval()
    print(f'g_loss: {g_loss:.4f}, d_loss: {d_loss:.4f}')
    z = torch.randn(9, 100).cuda()
    labels = torch.LongTensor(np.arange(9)).cuda()
    sample_images = generator(z, labels).unsqueeze(1).data.cpu()
    grid = make_grid(sample_images, nrow=3, normalize=True).permute(1,2,0).numpy()
    plt.imshow(grid)
    plt.show()

In [ ]:
z = torch.randn(100, 100).cuda()
labels = torch.LongTensor([i for _ in range(10) for i in range(10)]).cuda()
sample_images = generator(z, labels).unsqueeze(1).data.cpu()
grid = make_grid(sample_images, nrow=10, normalize=True).permute(1,2,0).numpy()
fig, ax = plt.subplots(figsize=(15,15))
ax.imshow(grid)
_ = plt.yticks([])
_ = plt.xticks(np.arange(15, 300, 30), ['0', '1', '2', '3',\
                                        '4', '5', '6', '7', '8',\
                                        '9'], rotation=45, fontsize=20)

Улучшилось ли качество генерируемых изображений?

In [ ]:
# your conclusions here

## Задание 2.1. Чувствительность к гиперпараметрам

 Одной из проблем при работе с GAN является очень высокая чувствительность к настройке гиперпараметров. Попробуйте для нашего простого GAN с функцией потерь LSGAN использовать оптимизаторы Adam с learning rate = 1e-4 и дефолтными скользящими параметрами betas.

Обучается ли модель?

In [ ]:
# your try here

# Задание 3. Deeply Convolutional GAN

В первом задании мы реализовали почти точную копию оригинальной GAN-сети. Однако эта архитектура не позволяет получить представления об изображении в пространстве пикселей. Она не способна учитывать такие вещи, как например "резкие края", потому что в ней отсутствуют какие-либо сверточные слои. Поэтому в этом разделе попробуем воссоздать архитектуру [DCGAN](https://arxiv.org/abs/1511.06434), где используются сверточные сети.


### Дискриминатор
Мы хотим присоединять метку класса к объекту. Не очевидным является то, как присоединить ээмбеддинг метки к объекту - картинке. Поэтому давайте разобьем архитектуру дискриминатора на 2 блока: CNN блок и full connected блок. После прогона объекта через CNN блок и разворотом во Flatten, будем добавлять метку и передавать далее в full connected блок.

CNN блок:
* Преобразовать "линейку", получаемую от генератора, в тензор изображения размера 1x28x28 (используйте nn.Unflatten)
* Conv2D, 32 фичи (выходных канала), ядро 5x5 с шагом (stride) 1, функция активации Leaky ReLU (alpha=0.01)
* MaxPool2D, ядро 2x2 с шагом 2
* Conv2D, 64 фичи, ядро 5x5 с шагом 1, Leaky ReLU (alpha=0.01)
* MaxPool2D, ядро 2x2 с шагом 2
* Flatten слой

Full connected блок:
* Полносвязный слой со входом 4 x 4 x 64 + 10 (посчитан из результата конволюций + размер меток) и таким же выходом, Leaky ReLU (alpha=0.01)
* Полносвязный слой с выходом  1

In [ ]:
class DiscriminatorV2(nn.Module):
    def __init__(self):
        set_random_seed(42)
        super().__init__()

        self.label_emb = nn.Embedding(10, 10)

        self.cnn = nn.Sequential(
            # your code here
        )

        self.fc = nn.Sequential(
            # your code here
        )

    def forward(self, x, labels):
        x = x.view(x.size(0), 784)
        c = self.label_emb(labels)
        x = self.cnn(x) # все конволюции и Flatten
        x = torch.cat([x, c], 1) # приклеиваем метку к объекту уже после прогона через CNN слой
        x = self.fc(x) # все после Flatten
        return x.squeeze()

### Генератор
Для генератора мы точно скопируем архитектуру из статьи [InfoGAN paper](https://arxiv.org/pdf/1606.03657.pdf)

* Полносвязный слой со входом 100 + 10 (шум + метки) и выходом 1024, ReLU
* BatchNorm1d
* Полносвязный слой с выходом 128 x 7 x 7, ReLU
* BatchNorm1d
* Разворот в тензор картинки 128x7x7 (nn.Unflatten)
* Обратная свертка ([nn.ConvTranspose2d](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d_transpose)), 64 фичи, ядро 4x4 с шагом 2 и 'same' падингом, ReLU
* BatchNorm2d
* Обратная свертка с 1 фичей, ядро 4x4 с шагом 2 и 'same' падингом, TanH
* Должна получиться картинка 28x28 с 1 каналом, разворачиваем в вектор длины 784 (Flatten слой)

In [ ]:
class GeneratorV2(nn.Module):
    def __init__(self, noise_dim=100):
        set_random_seed(42)
        super().__init__()

        self.label_emb = nn.Embedding(10, 10)

        self.model = nn.Sequential(
            # your code here
        )

    def forward(self, z, labels):
        c = self.label_emb(labels)
        x = torch.cat([z, c], 1)
        out = self.model(x)
        return out.view(x.size(0), 28, 28)

In [ ]:
generator = GeneratorV2().cuda()
discriminator = DiscriminatorV2().cuda()

In [ ]:
d_optimizer = # your code here
g_optimizer = # your code here

In [ ]:
num_epochs = 30
display_step = 300
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch}...')
    for i, (images, labels) in enumerate(data_loader):
        real_images = images.cuda()
        labels = labels.cuda()
        generator.train()
        batch_size = real_images.size(0)
        d_loss = ls_discriminator_train_step(len(real_images), discriminator,
                                          generator, d_optimizer, criterion,
                                          real_images, labels)


        g_loss = ls_generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion)

    generator.eval()
    print(f'g_loss: {g_loss:.4f}, d_loss: {d_loss:.4f}')
    z = torch.randn(9, 100).cuda()
    labels = torch.LongTensor(np.arange(9)).cuda()
    sample_images = generator(z, labels).unsqueeze(1).data.cpu()
    grid = make_grid(sample_images, nrow=3, normalize=True).permute(1,2,0).numpy()
    plt.imshow(grid)
    plt.show()

In [ ]:
z = torch.randn(100, 100).cuda()
labels = torch.LongTensor([i for _ in range(10) for i in range(10)]).cuda()
sample_images = generator(z, labels).unsqueeze(1).data.cpu()
grid = make_grid(sample_images, nrow=10, normalize=True).permute(1,2,0).numpy()
fig, ax = plt.subplots(figsize=(15,15))
ax.imshow(grid)
_ = plt.yticks([])
_ = plt.xticks(np.arange(15, 300, 30), ['0', '1', '2', '3',\
                                        '4', '5', '6', '7', '8',\
                                        '9'], rotation=45, fontsize=20)

Сравните результаты обучения двух архитектур (простой начальной и DCGAN). Какие выводы можно сделать?

In [ ]:
# your conclusions here